In [2]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import *
import numpy as np
import sys, os
os.chdir('..')
join = os.path.join(os.getcwd(), '_global')
sys.path.extend([join])
from _global.config import *
from _global.funcs import *
from _global.bbx import *
sys.path.extend([RESEARCH])
from object_detection.utils import config_util
from object_detection.builders import model_builder
from IPython import get_ipython
import cv2 
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)
ipython = get_ipython()

In [3]:
def _bytes_feature(value):
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() 
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [4]:
max_classes = 2

In [19]:
feature_des = {
        'image/filename': tf.io.VarLenFeature(dtype=tf.string),
        'image/encoded': tf.io.FixedLenFeature([], dtype=tf.string),
        'image/label':tf.io.FixedLenFeature([1],dtype=tf.int64),
        'image/labelname':tf.io.VarLenFeature(dtype=tf.string),
}
def parse_fn(proto):
    ex = tf.io.parse_single_example(proto, feature_des)
    image = tf.io.decode_jpeg(ex['image/encoded'], channels=3)
    image = tf.cast(image, tf.uint8)
    index =  tf.cast(ex['image/label'], tf.int64) -1
    labels = [1. if i == index else 0. for i in range(max_classes)]
    return image, tf.convert_to_tensor(labels, dtype=tf.float32)

In [20]:
def create_feature(img_bytes, relative_path, label_name, label_num):

    return tf.train.Example(
        features = tf.train.Features(feature = {
            'image/filename': tf.train.Feature(bytes_list = tf.train.BytesList(value = [bytes(relative_path, 'utf-8') ])),
            'image/encoded': _bytes_feature(img_bytes),
            'image/label':tf.train.Feature(int64_list = tf.train.Int64List(value = [label_num])),
            'image/labelname':tf.train.Feature(bytes_list = tf.train.BytesList(value = [bytes(label_name, 'utf-8') ])),
        })
    )

def parse_images(output_tfrecord_name,output_dir_path, label_name, label_num):
    with tf.io.TFRecordWriter(output_tfrecord_name) as writer:
        for file in os.listdir(output_dir_path):
            with tf.io.gfile.GFile(os.path.join(output_dir_path, file), 'rb') as fid:
                img_bytes = fid.read()

                writer.write(
                    tf.train.Example.SerializeToString(create_feature(img_bytes, file, label_name, label_num))
                )

In [21]:
output_path = os.path.join(INSTALLATION_PATH, "Niklas_Cropped_Faces")
output_path2 = os.path.join(INSTALLATION_PATH, "Kek_Cropped_Faces")
output_tfrecord = os.path.join(DATA_PATH, "temp_12111053.tfrecord")
output_tfrecord2 = os.path.join(DATA_PATH, "temp2_12111053.tfrecord")

In [22]:
parse_images(output_tfrecord, output_path, 'Niklasulls', 1)
parse_images(output_tfrecord2, output_path2, 'Kek',2)

In [15]:
index = 0
[1. if i == index else 0. for i in range(max_classes)]
for y in tf.compat.v1.python_io.tf_record_iterator(output_tfrecord):
    x = parse_fn(y)
    print(x)
    break

(<tf.Tensor: shape=(250, 200, 3), dtype=uint8, numpy=
array([[[ 4,  5,  7],
        [ 3,  4,  6],
        [ 3,  4,  6],
        ...,
        [ 5,  6,  8],
        [ 5,  6, 10],
        [ 4,  5,  9]],

       [[ 4,  5,  7],
        [ 4,  5,  7],
        [ 3,  4,  6],
        ...,
        [ 4,  5,  7],
        [ 4,  5,  9],
        [ 4,  5,  9]],

       [[ 6,  7,  9],
        [ 5,  6,  8],
        [ 5,  6,  8],
        ...,
        [ 4,  5,  7],
        [ 4,  5,  9],
        [ 5,  6, 10]],

       ...,

       [[ 4,  5,  7],
        [ 5,  6,  8],
        [ 5,  6,  8],
        ...,
        [31, 36, 39],
        [19, 24, 27],
        [10, 15, 18]],

       [[ 4,  5,  7],
        [ 5,  6,  8],
        [ 5,  6,  8],
        ...,
        [26, 31, 34],
        [18, 22, 25],
        [11, 15, 18]],

       [[ 4,  5,  7],
        [ 5,  6,  8],
        [ 5,  6,  8],
        ...,
        [23, 28, 31],
        [17, 21, 24],
        [12, 16, 19]]], dtype=uint8)>, <tf.Tensor: shape=(2,), dtype=float3

In [5]:
LAST_DENSE_BEFORE_OUTPUT_NEURONAS = 512

In [24]:

base_model = applications.VGG19(
    include_top=False,
    input_shape=(*RECOG_IMAGE_SIZES, 3),
    weights='imagenet'
)
base_model.trainable = False


model = Sequential(base_model)


model.add(layers.Conv2D(256, (4, 4), activation='relu', padding="same"))
model.add(layers.MaxPool2D(strides=(2, 2), padding="same"))
model.add(layers.Conv2D(128, (4,4), activation='relu', padding="same"))
model.add(layers.MaxPool2D(strides=(2, 2), padding="same"))
model.add(layers.Conv2D(256, (4,4), activation='relu', padding="same"))
model.add(layers.MaxPool2D(strides=(2, 2), padding="same"))
model.add(layers.Conv2D(128, (4,4), activation='relu', padding="same"))
model.add(layers.MaxPool2D(strides=(2, 2), padding="same"))
model.add(layers.Conv2D(512, (4,4), activation='relu', padding="same"))
model.add(layers.MaxPool2D(strides=(2, 2), padding="same"))
model.add(layers.Conv2D(1024, (4,4), activation='relu', padding="same"))
model.add(layers.MaxPool2D(strides=(2, 2), padding="same"))
model.add(layers.Conv2D(512, (4,4), activation='relu', padding="same"))
model.add(layers.MaxPool2D(strides=(2, 2), padding="same"))
model.add(layers.Conv2D(1024, (4,4), activation='relu', padding="same"))
model.add(layers.MaxPool2D(strides=(2, 2), padding="same"))
model.add(layers.Flatten())
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dense(LAST_DENSE_BEFORE_OUTPUT_NEURONAS, activation='relu'))


In [25]:
# # 10 because its amount of classes we have
model.add(layers.Dense(max_classes, activation='softmax'))
model.compile(optimizer='adam', loss=losses.CategoricalCrossentropy(from_logits=False), metrics='accuracy', run_eagerly=True) 

init_dataset = tf.data.TFRecordDataset([output_tfrecord,output_tfrecord2])
size = 1
init_dataset = init_dataset.map(parse_fn)
init_dataset = init_dataset.shuffle(330)
init_dataset = init_dataset.batch(4)
model.fit(init_dataset, epochs=3)
model.save(OUTPUT_FACES_RECOG)
# save model
# load model
# read weights of layer before last layer
# add new layer with the new count of classes
# init the new layer with weights of previous last layer
# predict -> |

Epoch 1/3
83/83 [==============================] - 18s 182ms/step - loss: 0.1812 - accuracy: 0.9576
Epoch 2/3
83/83 [==============================] - 14s 168ms/step - loss: 7.2248e-10 - accuracy: 1.0000
Epoch 3/3
83/83 [==============================] - 14s 167ms/step - loss: 7.2248e-10 - accuracy: 1.0000


In [ ]:
model.summary()

In [26]:
def new_face_added(model, face_name):
    curr_classes = model.layers[:].pop().output_shape[-1]
    
    prev_output_layer = model.layers.pop()
    prev_weights = prev_output_layer.get_weights()[0]
    bias = prev_weights[1]
    sum = 1
    for shape in prev_weights.shape:
        sum *= shape
    prev_weights = np.reshape(prev_weights, (sum))
    prev_weights = np.array(prev_weights.tolist().append(bias))
    max_classes += 1
    for _ in range(LAST_DENSE_BEFORE_OUTPUT_NEURONAS * (max_classes - curr_classes)):
        prev_weights = np.insert(prev_weights, 0, 0.0, 0)
    
    new_init_weights = tf.constant_initializer(prev_weights)
    new_output_layer = layers.Dense(max_classes, activation='softmax', kernel_initializer=new_init_weights) 
    model.add(new_output_layer)

    extend_label_map(max_classes, face_name)

    return model

In [27]:
def new_faces(dataset, face_name):
    model = new_face_added(models.load_model(OUTPUT_FACES_RECOG), face_name)
    model.fit(dataset, epochs=3)

In [28]:
dataset = tf.data.TFRecordDataset([FACES_RECOG_TEMP_TFRECORD])
dataset = dataset.map(parse_fn)
dataset = dataset.batch(4)

In [ ]:
# new_faces(dataset)

In [6]:
face_OUTPUT_FACES_RECOG = models.load_model(OUTPUT_FACES_RECOG)
config = config_util.get_configs_from_pipeline_file(PIPELINE_CONFIG_PATH)
configs = config_util.get_configs_from_pipeline_file(PIPELINE_CONFIG_PATH)
bbx_localizer = model_builder.build(model_config=config['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=bbx_localizer) 
ckpt.restore(os.path.join(MODEL, 'ckpt-12')).expect_partial()


In [7]:
def extend_label_map(label_num, label_name): 
    with open(FACES_RECOG_LABEL_MAP, 'a') as f:
        _str = "\r\nitem {\r\n\tid: %s,\r\n\tname: \"%s\"\r\n}".format(label_name, label_num)
        f.write(_str)

In [18]:
def predict_face(frame, localizer, detector):
    face, best_bbx = find_and_extract_face(localizer, frame, None, True)
    face = tf.convert_to_tensor(face,dtype=tf.uint8)
    face = tf.expand_dims(face, 0)
    label = detector.predict(face)
    label = np.squeeze(label,axis=0)
    max_label = 0
    for i in range(len(label)):
        if label[i] >= label[max_label]:
            max_label = i

    #max label +2 because now we work with indecies because we have to work with ids that start from 1 and id 1 is unknown
    if label[max_label] < .60:
        max_label = 1
    else: max_label +=2

    label_map = label_map_util.load_labelmap(FACES_RECOG_LABEL_MAP)

    categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=max_classes+1, use_display_name=True)

    category_index = label_map_util.create_category_index(categories)   

    x = np.array(1.0)

    img_with_bbxs = viz_utils.visualize_boxes_and_labels_on_image_array(fram- e,
                                                        boxes=np.array([best_bbx]),
                                                        classes=np.array([max_label]),
                                                        scores=np.array([1.0]),
                                                        category_index=category_index,
                                                        agnostic_mode=False,
                                                        skip_scores=True
                                                    )
    
    print(label)
    return img_with_bbxs


In [16]:
y = np.array([1.0])
for _ in y:
    print(_)

1.0


In [19]:
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
while cap.isOpened():
    ret, frame = cap.read()

    if ret == False:
        continue

    image_np_with_detections = predict_face(frame, bbx_localizer, face_OUTPUT_FACES_RECOG)
    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (width, height)))
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break

[0. 1.]
[0. 1.]
[1.0000000e+00 2.7528754e-22]
[1.0000000e+00 6.4017844e-19]
[1.0000000e+00 4.5841867e-24]
[1.0000000e+00 6.8329847e-28]
[1.0000000e+00 6.0516616e-19]
[1.000000e+00 8.394113e-27]
[1.0000000e+00 2.4817023e-12]
[1.000000e+00 1.757508e-21]
[1.0000000e+00 2.7661965e-26]
[1.0000000e+00 3.6210445e-22]
[1.0000000e+00 1.2973864e-25]
[1.0000000e+00 1.6111877e-28]
[1.000000e+00 5.541366e-32]
[1.0000000e+00 1.0026401e-36]
[1.0000000e+00 1.7409831e-32]
[1.000000e+00 6.701176e-37]
[1.0000000e+00 8.9519635e-35]
[1.0000000e+00 1.0155951e-33]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[1.0000000e+00 1.5568767e-35]
[1.0000000e+00 1.4059863e-17]
[1.0000000e+00 1.9775338e-11]
[1.000000e+00 8.694709e-10]
[1.0000000e+00 2.5240985e-23]
[1.0000000e+00 1.0880252e-18]
[1.0000000e+00 1.3426916e-18]
[1.00000e+00 7.74523e-15]
[1.0000000e+00 1.0286914e-27]
[1.0000000e+00 1.3762989e-28]
[1.0000000e+00 2.7371243e-31]
[1. 0.]
[1. 0.]
[1.00000e+00 4.00552e-28]
[0. 1.]
[1.0000000e+00 2.7859829e-38]
[1.00000